# 01 — BG Forecast Preprocessing

Loads the raw `main_dataset.csv` (event-based), cleans and processes it into a uniform 5-min time series per patient with glucose, insulin, carbs, IOB, COB. Then engineers features, builds sliding-window sequences, and saves `.npz` files for training.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib, os, json

## 1. Load Raw Data

In [2]:
raw = pd.read_csv('main_dataset.csv', low_memory=False)
print(f"Raw shape: {raw.shape}")
print(f"Event types: {raw['event_type'].unique()}")
print(f"Patients: {sorted(raw['patient_id'].unique())}")
raw.head()

Raw shape: (548092, 25)
Event types: ['glucose_level' 'finger_stick' 'basal' 'temp_basal' 'bolus' 'meal'
 'sleep' 'work' 'illness' 'exercise' 'basis_heart_rate' 'basis_gsr'
 'basis_skin_temperature' 'basis_air_temperature' 'basis_steps'
 'basis_sleep' 'stressors' 'hypo_event']
Patients: [np.int64(559), np.int64(563), np.int64(570), np.int64(575), np.int64(588), np.int64(591)]


,patient_id,weight,insulin_type,source_file,event_type,@ts,@value,@ts_begin,@ts_end,@type,...,@description,@duration,@competitive,@tbegin,@tend,start_time_raw,end_time_raw,start_time,end_time,symptom
0,559,99,Novalog,559-ws-testing.xml,glucose_level,18-01-2022 00:01:00,179.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18-01-2022 00:01:00,NaN,2022-01-18 00:01:00,NaN,NaN
1,559,99,Novalog,559-ws-testing.xml,glucose_level,18-01-2022 00:06:00,183.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18-01-2022 00:06:00,NaN,2022-01-18 00:06:00,NaN,NaN
2,559,99,Novalog,559-ws-testing.xml,glucose_level,18-01-2022 00:11:00,187.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18-01-2022 00:11:00,NaN,2022-01-18 00:11:00,NaN,NaN
3,559,99,Novalog,559-ws-testing.xml,glucose_level,18-01-2022 00:16:00,191.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18-01-2022 00:16:00,NaN,2022-01-18 00:16:00,NaN,NaN
4,559,99,Novalog,559-ws-testing.xml,glucose_level,18-01-2022 00:21:00,195.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18-01-2022 00:21:00,NaN,2022-01-18 00:21:00,NaN,NaN


## 2. Extract Glucose, Bolus, and Meal Events

We extract three key event types and parse their timestamps.

In [3]:
# --- Glucose readings ---
glucose_df = raw[raw['event_type'] == 'glucose_level'][['patient_id', '@ts', '@value']].copy()
glucose_df.rename(columns={'@ts': 'timestamp', '@value': 'glucose'}, inplace=True)
glucose_df['timestamp'] = pd.to_datetime(glucose_df['timestamp'], dayfirst=True)
glucose_df['glucose'] = pd.to_numeric(glucose_df['glucose'], errors='coerce')
glucose_df.dropna(subset=['glucose'], inplace=True)
print(f"Glucose events: {len(glucose_df)}")

# --- Bolus (insulin) events ---
bolus_df = raw[raw['event_type'] == 'bolus'][['patient_id', '@ts', '@dose']].copy()
bolus_df.rename(columns={'@ts': 'timestamp', '@dose': 'insulin'}, inplace=True)
bolus_df['timestamp'] = pd.to_datetime(bolus_df['timestamp'], dayfirst=True)
bolus_df['insulin'] = pd.to_numeric(bolus_df['insulin'], errors='coerce').fillna(0)
print(f"Bolus events: {len(bolus_df)}")

# --- Meal (carb) events ---
meal_df = raw[raw['event_type'] == 'meal'][['patient_id', '@ts', '@carbs']].copy()
meal_df.rename(columns={'@ts': 'timestamp', '@carbs': 'carbs'}, inplace=True)
meal_df['timestamp'] = pd.to_datetime(meal_df['timestamp'], dayfirst=True)
meal_df['carbs'] = pd.to_numeric(meal_df['carbs'], errors='coerce').fillna(0)
print(f"Meal events: {len(meal_df)}")

Glucose events: 85225
Bolus events: 1791
Meal events: 1303


## 3. Build Uniform 5-min Time Series Per Patient

For each patient:
1. Create a 5-min grid covering their glucose data range
2. Round glucose readings to nearest 5 min and merge
3. Interpolate missing glucose values
4. Map bolus and meal events to the nearest 5-min slot

In [4]:
patient_dfs = []

for pid in sorted(glucose_df['patient_id'].unique()):
    # --- Glucose grid ---
    g = glucose_df[glucose_df['patient_id'] == pid].copy()
    g['timestamp'] = g['timestamp'].dt.round('5min')
    # Average duplicates at same rounded timestamp
    g = g.groupby('timestamp')['glucose'].mean().reset_index()
    
    t_min, t_max = g['timestamp'].min(), g['timestamp'].max()
    grid = pd.DataFrame({'timestamp': pd.date_range(t_min, t_max, freq='5min')})
    grid = grid.merge(g, on='timestamp', how='left')
    
    # Interpolate glucose (linear), limit to 6 consecutive gaps (30 min)
    grid['glucose'] = grid['glucose'].interpolate(method='linear', limit=6)
    
    # --- Bolus ---
    b = bolus_df[bolus_df['patient_id'] == pid].copy()
    b['timestamp'] = b['timestamp'].dt.round('5min')
    b = b.groupby('timestamp')['insulin'].sum().reset_index()
    grid = grid.merge(b, on='timestamp', how='left')
    grid['insulin'] = grid['insulin'].fillna(0)
    
    # --- Meals ---
    m = meal_df[meal_df['patient_id'] == pid].copy()
    m['timestamp'] = m['timestamp'].dt.round('5min')
    m = m.groupby('timestamp')['carbs'].sum().reset_index()
    grid = grid.merge(m, on='timestamp', how='left')
    grid['carbs'] = grid['carbs'].fillna(0)
    
    grid['patient_id'] = pid
    
    # Drop rows where glucose is still NaN (edges beyond interpolation limit)
    grid.dropna(subset=['glucose'], inplace=True)
    
    patient_dfs.append(grid)
    print(f"Patient {pid}: {len(grid)} rows, "
          f"{g['glucose'].notna().sum()} raw glucose, "
          f"{len(b)} bolus events, {len(m)} meal events")

df = pd.concat(patient_dfs, ignore_index=True)
print(f"\nCombined shape: {df.shape}")
df.head()

Patient 559: 13607 rows, 13310 raw glucose, 0 bolus events, 179 meal events
Patient 563: 14804 rows, 14694 raw glucose, 0 bolus events, 155 meal events
Patient 570: 13887 rows, 13727 raw glucose, 0 bolus events, 169 meal events
Patient 575: 14856 rows, 14456 raw glucose, 0 bolus events, 288 meal events
Patient 588: 15503 rows, 15431 raw glucose, 0 bolus events, 257 meal events
Patient 591: 13761 rows, 13607 raw glucose, 0 bolus events, 253 meal events

Combined shape: (86418, 5)


,timestamp,glucose,insulin,carbs,patient_id
0,2021-12-07 01:15:00,101.0,0.0,0.0,559
1,2021-12-07 01:20:00,98.0,0.0,0.0,559
2,2021-12-07 01:25:00,104.0,0.0,0.0,559
3,2021-12-07 01:30:00,112.0,0.0,0.0,559
4,2021-12-07 01:35:00,120.0,0.0,0.0,559


## 4. Compute IOB and COB

- **IOB** (Insulin on Board): exponential decay of past insulin doses, ~4 hour action time
- **COB** (Carbs on Board): exponential decay of past carb intake, ~3 hour absorption time

In [7]:
# Decay constants (per 5-min step)
IOB_HALF_LIFE_MIN = 75   # insulin half-life ~75 min
COB_HALF_LIFE_MIN = 45   # carb absorption half-life ~45 min

iob_decay = 0.5 ** (5 / IOB_HALF_LIFE_MIN)
cob_decay = 0.5 ** (5 / COB_HALF_LIFE_MIN)
print(f"IOB decay per step: {iob_decay:.4f}, COB decay per step: {cob_decay:.4f}")

iob_list = []
cob_list = []

for pid in sorted(df['patient_id'].unique()):
    mask = df['patient_id'] == pid
    insulin_vals = df.loc[mask, 'insulin'].values
    carb_vals = df.loc[mask, 'carbs'].values
    
    iob = np.zeros(len(insulin_vals))
    cob = np.zeros(len(carb_vals))
    
    for i in range(len(insulin_vals)):
        iob[i] = (iob[i-1] * iob_decay if i > 0 else 0) + insulin_vals[i]
        cob[i] = (cob[i-1] * cob_decay if i > 0 else 0) + carb_vals[i]
    
    iob_list.append(iob)
    cob_list.append(cob)

df['IOB'] = np.concatenate(iob_list)
df['COB'] = np.concatenate(cob_list)

print(f"IOB range: {df['IOB'].min():.2f} – {df['IOB'].max():.2f}")
print(f"COB range: {df['COB'].min():.2f} – {df['COB'].max():.2f}")
df[['glucose', 'insulin', 'carbs', 'IOB', 'COB']].describe()

IOB decay per step: 0.9548, COB decay per step: 0.9259
IOB range: 0.00 – 0.00
COB range: 0.00 – 450.14


,glucose,insulin,carbs,IOB,COB
count,86418.000000,86418.0,86418.000000,86418.0,86418.000000
mean,162.019856,0.0,0.619258,0.0,8.346355
std,60.934943,0.0,6.670302,0.0,16.246173
min,40.000000,0.0,0.000000,0.0,0.000000
25%,115.000000,0.0,0.000000,0.0,0.106355
50%,155.000000,0.0,0.000000,0.0,1.654526
75%,202.000000,0.0,0.000000,0.0,9.497555
max,400.000000,0.0,450.000000,0.0,450.141240


## 5. Feature Engineering

In [ ]:
# Time-of-day cyclic encoding
hour_frac = df['timestamp'].dt.hour + df['timestamp'].dt.minute / 60.0
df['hour_sin'] = np.sin(2 * np.pi * hour_frac / 24)
df['hour_cos'] = np.cos(2 * np.pi * hour_frac / 24)

# Glucose rate of change (backward difference per patient)
df['glucose_roc'] = df.groupby('patient_id')['glucose'].diff().fillna(0)

# Features to use as model input
FEATURE_COLS = ['glucose', 'insulin', 'carbs', 'IOB', 'COB',
                'hour_sin', 'hour_cos', 'glucose_roc']

print(f"Feature columns ({len(FEATURE_COLS)}): {FEATURE_COLS}")
df[FEATURE_COLS].describe()

Feature columns (8): ['glucose', 'insulin', 'carbs', 'IOB', 'COB', 'hour_sin', 'hour_cos', 'glucose_roc']


,glucose,insulin,carbs,IOB,COB,hour_sin,hour_cos,glucose_roc
count,86418.000000,86418.0,86418.000000,86418.0,86418.000000,86418.000000,8.641800e+04,86418.000000
mean,162.019856,0.0,0.619258,0.0,8.346355,-0.010853,3.272199e-03,0.003969
std,60.934943,0.0,6.670302,0.0,16.246173,0.706085,7.080443e-01,6.490869
min,40.000000,0.0,0.000000,0.0,0.000000,-1.000000,-1.000000e+00,-243.460000
25%,115.000000,0.0,0.000000,0.0,0.106355,-0.722364,-7.071068e-01,-3.000000
50%,155.000000,0.0,0.000000,0.0,1.654526,-0.021815,6.123234e-17,0.000000
75%,202.000000,0.0,0.000000,0.0,9.497555,0.691513,7.071068e-01,2.000000
max,400.000000,0.0,450.000000,0.0,450.141240,1.000000,1.000000e+00,243.666667


## 6. Build Sliding-Window Sequences

- **Input window**: 12 timesteps (60 min at 5-min intervals)
- **Targets**: glucose at t+6 (30 min), t+12 (60 min), t+18 (90 min)
- Sequences never cross patient boundaries

In [9]:
INPUT_LEN = 12   # 60 min
HORIZONS = [6, 12, 18]  # 30, 60, 90 min ahead
MAX_HORIZON = max(HORIZONS)

def build_sequences(patient_df):
    """Build (X, y) sliding windows for a single patient."""
    features = patient_df[FEATURE_COLS].values
    glucose = patient_df['glucose'].values
    X_list, y_list = [], []
    for i in range(len(features) - INPUT_LEN - MAX_HORIZON):
        X_list.append(features[i : i + INPUT_LEN])
        targets = [glucose[i + INPUT_LEN + h - 1] for h in HORIZONS]
        y_list.append(targets)
    return np.array(X_list), np.array(y_list)

# Build per patient
all_X, all_y, all_pid = [], [], []
for pid, grp in df.sort_values(['patient_id', 'timestamp']).groupby('patient_id'):
    X_p, y_p = build_sequences(grp.reset_index(drop=True))
    all_X.append(X_p)
    all_y.append(y_p)
    all_pid.extend([pid] * len(X_p))
    print(f"Patient {pid}: {len(X_p)} sequences")

X_all = np.concatenate(all_X)
y_all = np.concatenate(all_y)
pid_arr = np.array(all_pid)
print(f"\nTotal: X={X_all.shape}, y={y_all.shape}")

Patient 559: 13577 sequences
Patient 563: 14774 sequences
Patient 570: 13857 sequences
Patient 575: 14826 sequences
Patient 588: 15473 sequences
Patient 591: 13731 sequences

Total: X=(86238, 12, 8), y=(86238, 3)


## 7. Patient-Level Train / Val / Test Split

Split patients so there is no data leakage between sets.

In [10]:
patients = sorted(df['patient_id'].unique())
print(f"All patients: {patients}")

# With 6 patients: 4 train, 1 val, 1 test
np.random.seed(42)
np.random.shuffle(patients)
train_pids = patients[:4]
val_pids   = patients[4:5]
test_pids  = patients[5:6]
print(f"Train: {train_pids}, Val: {val_pids}, Test: {test_pids}")

train_mask = np.isin(pid_arr, train_pids)
val_mask   = np.isin(pid_arr, val_pids)
test_mask  = np.isin(pid_arr, test_pids)

X_train, y_train = X_all[train_mask], y_all[train_mask]
X_val,   y_val   = X_all[val_mask],   y_all[val_mask]
X_test,  y_test  = X_all[test_mask],  y_all[test_mask]

print(f"Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}")

All patients: [np.int64(559), np.int64(563), np.int64(570), np.int64(575), np.int64(588), np.int64(591)]
Train: [np.int64(559), np.int64(563), np.int64(591), np.int64(570)], Val: [np.int64(588)], Test: [np.int64(575)]
Train: 55939, Val: 15473, Test: 14826


## 8. Fit Scaler on Training Data Only

In [11]:
# Reshape to 2D for scaler, then back to 3D
n_features = X_train.shape[2]

scaler = StandardScaler()
scaler.fit(X_train.reshape(-1, n_features))

def scale_3d(X, scaler):
    orig_shape = X.shape
    return scaler.transform(X.reshape(-1, orig_shape[2])).reshape(orig_shape)

X_train_s = scale_3d(X_train, scaler)
X_val_s   = scale_3d(X_val,   scaler)
X_test_s  = scale_3d(X_test,  scaler)

print(f"Scaled X_train mean (should be ~0): {X_train_s.reshape(-1, n_features).mean(axis=0).round(2)}")

Scaled X_train mean (should be ~0): [ 0.  0. -0.  0. -0. -0. -0. -0.]


## 9. Save Artifacts

In [12]:
os.makedirs('processed', exist_ok=True)

np.savez('processed/bg_forecast_sequences.npz',
         X_train=X_train_s, y_train=y_train,
         X_val=X_val_s,     y_val=y_val,
         X_test=X_test_s,   y_test=y_test)

joblib.dump(scaler, 'processed/bg_forecast_scaler.joblib')

# Save metadata for the backend module
meta = {'feature_cols': FEATURE_COLS, 'input_len': INPUT_LEN, 'horizons': HORIZONS}
with open('processed/bg_forecast_meta.json', 'w') as f:
    json.dump(meta, f)

print('Saved: processed/bg_forecast_sequences.npz')
print('Saved: processed/bg_forecast_scaler.joblib')
print('Saved: processed/bg_forecast_meta.json')

Saved: processed/bg_forecast_sequences.npz
Saved: processed/bg_forecast_scaler.joblib
Saved: processed/bg_forecast_meta.json
